# Notebook 1: Data Cleaning, Merging, and Exportation

In this notebook, we explore the structure and content of our datasets, and we conduct data manipulation to clean the data. At the end, we export a cleaned and merged version of our datasets to a CSV file for us to use in our other notebooks for analysis and visualization.

## 1.0. Project Dependencies

### 1.1. Pip Installs

In [1]:
# Running Python 3.12; None Needed

### 1.2. Import Libraries

In [2]:
import pandas as pd

### 1.3. Import the Datasets

#### 1.3.1. Import District Expenses Data

In [3]:
# Read in the expense data from the Excel file, skipping the first 10 rows of header data 
expense_df = pd.read_excel('current_expense.xlsx', skiprows=10)

# Rename the columns to be more descriptive and drop the 'CO' column
expense_df = expense_df.rename(columns={'CDS':'District Code',
                                        'District': 'District Name',
                                        'Current\nExpense ADA': 'Expense ADA',
                                        'Current\nExpense per ADA': 'Expense per ADA'})
expense_df = expense_df.drop(['CO'], axis=1)

# Display the cleaned expense data
display(expense_df.shape)
display(expense_df.head())

(933, 6)

,District Code,District Name,EDP 365,Expense ADA,Expense per ADA,LEA Type
0,61119,Alameda Unified,1.550948e+08,8567.86,18101.93,Unified
1,61127,Albany City Unified,6.149090e+07,3435.41,17899.14,Unified
2,61143,Berkeley Unified,2.205508e+08,8572.17,25728.70,Unified
3,61150,Castro Valley Unified,1.424913e+08,8991.52,15847.30,Unified
4,61168,Emery Unified,1.586300e+07,554.70,28597.44,Unified


#### 1.3.2. Import District Demographic Data

In [4]:
# Read in the district data from the Excel file
district_df = pd.read_excel('district_areas.xlsx')

# Convert the 'District Code' column to a string, extracts the last 5 characters, 
# converts it back to an integer, and updates the column with this new value.
district_df['District Code'] = district_df['District Code'].astype(str).str[-5:].astype('int64')

# Drop unnecessary columns from the DataFrame
district_df = district_df.drop(['Year',
                                'DistrctAreaSqMi',
                                'Shape__Area',
                                'Shape__Length',
                                'OBJECTID',
                                'US Congress District',
                                'CA Senate District',
                                'CA Assembly District',
                                'Update Notes'], axis=1)

# Display the cleaned district data
display(district_df.shape)
display(district_df.head())

(938, 45)

,Fed ID,District Code,CDS Code,County Name,District Name,District Type,Grade Low,Grade High,Grade Low Census,Grade High Census,...,Foster,Foster (%),Homeless,Homeless (%),Migrant,Migrant (%),Students with Disabilities,Students with Disabilities (%),Socioeconomically Disadvantaged,Socioeconomically Disadvantaged (%)
0,601770,61119,1611190000000,Alameda,Alameda Unified,Unified,KG,12,KG,12,...,27,0.3,91,0.9,0,0.0,1286,12.2,4035,38.2
1,601860,61127,1611270000000,Alameda,Albany City Unified,Unified,KG,12,KG,12,...,1,0.0,24,0.7,0,0.0,320,9.0,1122,31.4
2,604740,61143,1611430000000,Alameda,Berkeley Unified,Unified,KG,12,KG,12,...,32,0.4,150,1.7,0,0.0,1092,12.0,2508,27.6
3,607800,61150,1611500000000,Alameda,Castro Valley Unified,Unified,KG,12,KG,12,...,5,0.1,135,1.4,0,0.0,1048,11.0,3686,38.8
4,612630,61168,1611680000000,Alameda,Emery Unified,Unified,KG,12,KG,12,...,1,0.2,13,2.2,0,0.0,75,12.5,327,54.5


## 2.0. Data Cleaning

### 2.1. Check the expense_df and district_df DataFrames for Column Types & Missing (NaN) Values

In [5]:
display(expense_df.info())
display(district_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 933 entries, 0 to 932
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   District Code    933 non-null    int64  
 1   District Name    933 non-null    object 
 2   EDP 365          933 non-null    float64
 3   Expense ADA      933 non-null    float64
 4   Expense per ADA  933 non-null    float64
 5   LEA Type         933 non-null    object 
dtypes: float64(3), int64(1), object(2)
memory usage: 43.9+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 938 entries, 0 to 937
Data columns (total 45 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Fed ID                               938 non-null    int64  
 1   District Code                        938 non-null    int64  
 2   CDS Code                             938 non-null    int64  
 3   County Name                          938 non-null    object 
 4   District Name                        938 non-null    object 
 5   District Type                        938 non-null    object 
 6   Grade Low                            938 non-null    object 
 7   Grade High                           938 non-null    int64  
 8   Grade Low Census                     938 non-null    object 
 9   Grade High Census                    938 non-null    int64  
 10  Assistance Status                    930 non-null    object 
 11  Locale                          

None

### 2.2. Determining District Differences Between the expense_df and district_df DataFrames

In [6]:
# Create a set of district codes from both 'expense_df' and 'district_df' DataFrames
set_district = set(district_df['District Code'])
set_expense = set(expense_df['District Code'])

# Find codes that are in 'district_df' (938 rows) and NOT in 'expense_df' (933 rowws)
codes_excluded_from_expense = set_district - set_expense

# Find codes that are in 'expense_df' (933 rows) and NOT in 'district_df'(938 rows)
codes_excluded_from_district = set_expense - set_district

# Check excluded codes between the two datasets
print(f'Number of Values not in Expense: {len(codes_excluded_from_expense)} \nList: {codes_excluded_from_expense}')
print(district_df[district_df['District Code'].isin(codes_excluded_from_expense)]['District Name'].tolist())
print(f'Number of Values not in District: {len(codes_excluded_from_district)} \nList: {codes_excluded_from_district}')
print(expense_df[expense_df['District Code'].isin(codes_excluded_from_district)]['District Name'].tolist())

Number of Values not in Expense: 10 
List: {70912, 70854, 71175, 70920, 71167, 70862, 65557, 69815, 65599, 69823}
['Arena Union Elementary', 'Point Arena Joint Union High', 'Santa Cruz City Elementary', 'Santa Cruz City High', 'Petaluma City Elementary', 'Petaluma Joint Union High', 'Santa Rosa Elementary', 'Santa Rosa High', 'Modesto City Elementary', 'Modesto City High']
Number of Values not in District: 5 
List: {76349, 40261, 40717, 40246, 40253}
['Arena Union Elementary/Point Arena Joint Union High ', 'Santa Cruz City Elementary/High', 'Petaluma City Elementary/Joint Union High', 'Santa Rosa City Schools', 'Modesto City Schools']


### 2.3. Consolidating Non-Unifed Districts in district_df into Unified Districts, as Shown in expense_df

We identified 10 districts in our `district_df` dataset that were not included in our `expense_df` dataset.

>`['Arena Union Elementary', 'Point Arena Joint Union High', 'Santa Cruz City Elementary', 'Santa Cruz City High', 'Petaluma City Elementary', 'Petaluma Joint Union High', 'Santa Rosa Elementary', 'Santa Rosa High', 'Modesto City Elementary', 'Modesto City High']`

Likewise, 5 districts in our `expense_df` dataset were not in our `district_df` dataset.

>`['Arena Union Elementary/Point Arena Joint Union High ', 'Santa Cruz City Elementary/High', 'Petaluma City Elementary/Joint Union High', 'Santa Rosa City Schools', 'Modesto City Schools']`

After comparing the two lists, we found the following:

In [7]:
# Create a dictionary to map the non-unified district names to their unified names
district_table_dict = {
    'Arena Union Elementary': 'Arena Union Elementary/Point Arena Joint Union High ',
    'Point Arena Joint Union High': 'Arena Union Elementary/Point Arena Joint Union High ',
    'Santa Cruz City Elementary': 'Santa Cruz City Elementary/High',
    'Santa Cruz City High': 'Santa Cruz City Elementary/High',
    'Petaluma City Elementary': 'Petaluma City Elementary/Joint Union High',
    'Petaluma Joint Union High': 'Petaluma City Elementary/Joint Union High',
    'Santa Rosa Elementary': 'Santa Rosa City Schools',
    'Santa Rosa High': 'Santa Rosa City Schools',
    'Modesto City Elementary': 'Modesto City Schools',
    'Modesto City High': 'Modesto City Schools'
}

# Create a DataFrame from the dictionary to display the mapping of non-unified to unified district names
dict_df = (
    pd.DataFrame(list(district_table_dict.items()), columns=['Original, Non-Unified Districts', 'Unified District Name'])
    .groupby('Unified District Name')['Original, Non-Unified Districts']
    .apply(lambda x: ', '.join(x))
    .reset_index()
)

# Display the DataFrame
display(dict_df)

,Unified District Name,"Original, Non-Unified Districts"
0,Arena Union Elementary/Point Arena Joint Union...,"Arena Union Elementary, Point Arena Joint Unio..."
1,Modesto City Schools,"Modesto City Elementary, Modesto City High"
2,Petaluma City Elementary/Joint Union High,"Petaluma City Elementary, Petaluma Joint Union..."
3,Santa Cruz City Elementary/High,"Santa Cruz City Elementary, Santa Cruz City High"
4,Santa Rosa City Schools,"Santa Rosa Elementary, Santa Rosa High"


### 2.4. Plan to Maintain Fidelity with 5 Unified Districts

Given that the `expense_df` data used only the Unified District Names, we decided to do the following:

1. Manually identify all 10 districts in the `district_df` dataframe
2. `'Locale'` values were the same across unified districts, so we kept them as is.
3. Sum the values for the following district values:
>`['Enroll Total',  'Enroll Charter', 'Enroll Non Charter']`
4. Sum the values for all:
>`['Race & Ethnicity', 'English Learners', 'Foster, 'Homeless', 'Migrant', 'Students with Disabilities', and 'Socioeconomically Disadvantaged']` columns for Count values only.
5. Recalculate the percentages for all values in **Step 4**.

Ultimately, our final merged dataset will **only** contain the unified `'District Name'` values for these 10 specific districts.

### 2.5. Manally Merging 10 Districts into 5 Unified Districts & Merging District Expense & District Areas Datasets

In [8]:
# Display the number of unique district codes in both DataFrames
print('Unique District Codes in district_df: ', district_df['District Code'].nunique())
print('Unique District Codes in expense_df: ', expense_df['District Code'].nunique())

print('\n')


# Display the excluded district codes in both DataFrames
print('5 Codes Excluded from district_df: ', codes_excluded_from_district)
print('10 Codes Excluded from expense_df: ',codes_excluded_from_expense)

Unique District Codes in district_df:  938
Unique District Codes in expense_df:  933


5 Codes Excluded from district_df:  {76349, 40261, 40717, 40246, 40253}
10 Codes Excluded from expense_df:  {70912, 70854, 71175, 70920, 71167, 70862, 65557, 69815, 65599, 69823}


#### 2.5.1. Creating a Dictionary of `District Code`'s to be Unified & Analyzing `District Name`'s

In [9]:
# Key:    Un-unified District Code to be replaced
# Value:  Unified District Code to replace with
dict_of_codes = {
    65557: 76349,
    65599: 76349,
    69815: 40261,
    69823: 40261,
    70854: 40246,
    70862: 40246,
    70912: 40253,
    70920: 40253,
    71167: 40717,
    71175: 40717
}

# Create a copy of the 'district_df' DataFrame
district_df = district_df.copy()

# Replace the 10 District Codes in `district_df` with the 5 Unified District Codes from `expense_df`
district_df['District Code'] = district_df['District Code'].replace(dict_of_codes)

# Display the number of unique district codes in the updated 'district_df'
print("Unique Codes in district_df after 'District Code' replacement:", district_df['District Code'].nunique())

# Display the number of unique district **NAMES** in the 'district_df' and 'expense_df' DataFrames
print("Unique Names in district_df before 'District Name' replace: ", district_df['District Name'].nunique()) 
print("Unique Names in expense_df before 'District Name' replace: ", expense_df['District Name'].nunique())

Unique Codes in district_df after 'District Code' replacement: 933
Unique Names in district_df before 'District Name' replace:  927
Unique Names in expense_df before 'District Name' replace:  919


This tells us a few things:
1. *Number of unique*  `'District Code'` *values* > *Number of unique*  `'District Name'` *values*
2. `district_df` (927) **has 8 more unique `'District Name'` values** than `expense_df` does (919)

3. `district_df`: $$933 - 927 = 6$$ “collisions” (situations where 2+ codes share the same name)

4. `expense_df`: $$933 - 919 = 14$$ "collisions"

So *Number of unique*  `'District Code'` *values* > *Number of unique*  `'District Name'` *values* makes sense

#### 2.5.2. Determine Different District Names in `'expense_df'` & `'district_df'` DataFrames

In [10]:
# Create two variables to hold the unique district names from both DataFrames
names_in_district = set(district_df['District Name'].unique())
names_in_expense  = set(expense_df['District Name'].unique())

# Find the names that are in one DataFrame but not the other and store them in variables
only_in_district = names_in_district - names_in_expense
only_in_expense  = names_in_expense - names_in_district

# Print the number of names in each variable and the names themselves in alphabetical order
print("Names only in district_df:", len(only_in_district))
print(sorted(list(only_in_district)))
print("Names only in expense_df:", len(only_in_expense))
print(sorted(list(only_in_expense)))

Names only in district_df: 92
['Alisal Union', 'Arena Union Elementary', 'Arvin Union', 'Bakersfield City', 'Bellevue Union', 'Bolinas-Stinson Union', 'Brentwood Union', 'Cajon Valley Union', 'Cambrian', 'Campbell Union', 'Castaic Union', 'Chatom Union', 'Chualar Union', 'Columbia Union', 'Cupertino Union', 'El Monte City', 'Escondido Union', 'Eureka City Schools', 'Eureka Union', 'Evergreen Union', 'Hacienda la Puente Unified', 'Hawthorne', 'Hollister', 'Kenwood', 'Keyes Union', 'King City Union', 'Lakeside Joint', 'Lakeside Union', 'Latrobe', 'Lemon Grove', 'Lennox', 'Livingston Union', 'Los Nietos', 'Lowell Joint', 'Luther Burbank', 'McCabe Union Elementary', 'McFarland Unified', 'McKinleyville Union Elementary', 'McKittrick Elementary', 'Meadows Union', 'Menifee Union', 'Modesto City Elementary', 'Modesto City High', 'Moreland', 'Mountain View Whisman', 'Newhall', 'Nicasio', 'Nuview Union', 'Ocean View', 'Old Adobe Union', 'Ontario-Montclair', 'Orange Center', 'Orinda Union', 'Oro 

To summarize:
1. `expense_df` names are more longer/descriptive than `district_df` names because they are Unified names and contain the descriptive elementary/middle/high names 
2. This makes sense because `district_df` puts descriptive elementary/middle/high names in the `'District Type'` column

We will be using use `expense_df`'s `'District Name'` values as our "standardized district names"; as we are already using `expense_df`'s `District Code` values as our "standardized district codes".

#### 2.5.3. Determine the Non-Unique District Names

In [11]:
# Test for dupes with district_df
dupes = (
    district_df.groupby('District Name')['District Code']
    .nunique()
    .sort_values(ascending=False)
)

# Display the non-unique district names and their counts
print(dupes[dupes > 1])

District Name
Jefferson Elementary         3
Washington Unified           2
Pacific Union Elementary     2
Lakeside Union Elementary    2
Ocean View                   2
Liberty Elementary           2
Junction Elementary          2
Hope Elementary              2
Pioneer Union Elementary     2
Mountain View Elementary     2
Name: District Code, dtype: int64


To summarize the information above: There are $$2+1+1+1+1+1+1+1+1+1 = 11$$ "extra" `'District Code'` values

This means that there *11* non-unique `'District Name'` values. In other words, the same `'District Name'` (str) is tied to multiple `'District Code'` (int) values. Since we've already established *# of unique*  `'District Code'` *values* > *# of unique*  `'District Name'` *values*, we know that some names are used for multiple codes. In other words, `'District Name'` values are **non-unique**.

>> For example, `'District Name'` = `'Jefferson Elementary'` is tied to 3 `'District Code'` values (and counties): San Benito, San Joaquin, & San Mateo


We also know that `district_df` has a `'County Name'` column, which is important for visualization labels.
>>`expense_df` does not have a `'County Name'` column, so the duped/identical `'District Name'` values in `expense_df` will give us confusing visualizations.

In other words:
>`'District Code'` = Unique Authoritative Key

>`'District Name'` = Non-Unique, Descriptive Key

Therefore, going forward:
1. **For Analysis & Grouping**: We should always reference to each row of data by `'District Code'` because codes are unique.
2. **For Visualizations**: We should use the new `'District Label'` column, which will be a unique concatenation of "District Name (County)".

#### 2.5.4. Concatenate the Unified Districts

We've already replaced the 10 non-unified district codes with the 5 unified district codes.

So let's first take care of the data for these 10 rows before we do our final merge of `district_df` & `expense_df`.

In [12]:
# Create copies of the 'expense_df' and 'district_df' DataFrames to avoid modifying the original data
expense_data = expense_df.copy()
district_data = district_df.copy()

# Display the shape of the new 'expense_data' and 'district_data' DataFrames
print(expense_data.shape)
print(district_data.shape)

# Check for duplicate 'District Code' values in the 'district_data' DataFrame
dupes_2 = district_data['District Code'].value_counts()
print(dupes_2[dupes_2 > 1])

(933, 6)
(938, 45)
District Code
76349    2
40261    2
40253    2
40246    2
40717    2
Name: count, dtype: int64


In [13]:
list1 = dupes_2[dupes_2 > 1].index.tolist()
list1

[76349, 40261, 40253, 40246, 40717]

#### 2.5.5. Pair Up the Unified Districts and Ascertain dtypes

In [14]:
# Store the 5 unified district codes in a list
unified_list = [76349, 40261, 40246, 40253, 40717]

# Create new DataFrames containing the rows for each unified district code
unified_pair_1 = district_data[district_data['District Code'] == 76349]
unified_pair_2 = district_data[district_data['District Code'] == 40261]
unified_pair_3 = district_data[district_data['District Code'] == 40246]
unified_pair_4 = district_data[district_data['District Code'] == 40253]
unified_pair_5 = district_data[district_data['District Code'] == 40717]

# Store the list of DataFrames in a list variable
unified_pair_list = [unified_pair_1, unified_pair_2, unified_pair_3, unified_pair_4, unified_pair_5]

# Display the first DataFrame and its data types
display(unified_pair_1)
print(unified_pair_1.dtypes)    # Checked: All columns with numeric values are int64 or float64
district_table_dict

,Fed ID,District Code,CDS Code,County Name,District Name,District Type,Grade Low,Grade High,Grade Low Census,Grade High Census,...,Foster,Foster (%),Homeless,Homeless (%),Migrant,Migrant (%),Students with Disabilities,Students with Disabilities (%),Socioeconomically Disadvantaged,Socioeconomically Disadvantaged (%)
349,603090,76349,23655570000000,Mendocino,Arena Union Elementary,Elementary,KG,12,KG,8,...,1,0.3,16,5.6,0,0.0,37,12.9,160,55.7
353,631230,76349,23655990000000,Mendocino,Point Arena Joint Union High,High,09,12,09,12,...,3,2.1,25,17.5,2,1.4,27,18.9,83,58.0


Fed ID                                   int64
District Code                            int64
CDS Code                                 int64
County Name                             object
District Name                           object
District Type                           object
Grade Low                               object
Grade High                               int64
Grade Low Census                        object
Grade High Census                        int64
Assistance Status                       object
Locale                                  object
Enroll Total                             int64
Enroll Charter                           int64
Enroll Non Charter                       int64
African American                         int64
African American (%)                   float64
American Indian                          int64
American Indian (%)                    float64
Asian                                    int64
Asian (%)                              float64
Filipino     

{'Arena Union Elementary': 'Arena Union Elementary/Point Arena Joint Union High ',
 'Point Arena Joint Union High': 'Arena Union Elementary/Point Arena Joint Union High ',
 'Santa Cruz City Elementary': 'Santa Cruz City Elementary/High',
 'Santa Cruz City High': 'Santa Cruz City Elementary/High',
 'Petaluma City Elementary': 'Petaluma City Elementary/Joint Union High',
 'Petaluma Joint Union High': 'Petaluma City Elementary/Joint Union High',
 'Santa Rosa Elementary': 'Santa Rosa City Schools',
 'Santa Rosa High': 'Santa Rosa City Schools',
 'Modesto City Elementary': 'Modesto City Schools',
 'Modesto City High': 'Modesto City Schools'}

#### 2.5.6. Categorize the Different Columns

We need to determine what to do with each of the columns for each of the unified district pairs.

In [15]:
# Global Variables for use in next code block
KEEP_AS_IS_COLUMNS = [
    'District Code',
    'County Name',
    'District Type',
    'Grade Low',
    'Grade High',
    'Grade Low Census',
    'Grade High Census',
    'Assistance Status'
]

DELETE_COLUMNS = [
    'Fed ID',
    'CDS Code'
]

COUNT_COLUMNS = [
    'Enroll Total',
    'Enroll Charter',
    'Enroll Non Charter',
    'African American',
    'American Indian',
    'Asian',
    'Filipino',
    'Hispanic',
    'Pacific Islander',
    'White',
    'Two or More Races',
    'Not Reported',
    'English Learner',
    'Foster',
    'Homeless',
    'Migrant',
    'Students with Disabilities',
    'Socioeconomically Disadvantaged']

PERCENT_COLUMNS_DICT = {
    'African American (%)': 'African American',
    'American Indian (%)': 'American Indian',
    'Asian (%)': 'Asian',
    'Filipino (%)': 'Filipino',
    'Hispanic (%)': 'Hispanic',
    'Pacific Islander (%)': 'Pacific Islander',
    'White (%)': 'White',
    'Two or More Races (%)': 'Two or More Races',
    'Not Reported (%)': 'Not Reported',
    'English Learner (%)': 'English Learner',
    'Foster (%)': 'Foster',
    'Homeless (%)': 'Homeless',
    'Migrant (%)': 'Migrant',
    'Students with Disabilities (%)': 'Students with Disabilities',
    'Socioeconomically Disadvantaged (%)': 'Socioeconomically Disadvantaged'
}

#### 2.5.7. Write a Function to Merge the Unified Pairs & Run the Function

In [78]:
def merge_unified_pairs(
    unified_pair: pd.DataFrame,
    keep_cols: list[str],
    delete_cols: list[str],
    count_cols: list[str],
    percent_cols_dict: dict[str, str],
    district_table_dict: dict[str, str]
) -> pd.DataFrame:
    """
    Combine two district rows into one unified row.

    Args:
        unified_pair (pd.DataFrame): DataFrame containing unified pair data.
        keep_cols (list): Columns to keep as-is
        delete_cols (list): Columns to drop
        count_cols (list): Columns to sum
        percent_cols_dict (dict): Mapping of percentage columns to count columns
        district_table_dict (dict): Mapping of non-unified to unified district names

    Returns:
        pd.DataFrame: Processed DataFrame with summed counts, percentages, and labels.
    
    Steps:
    1. Drop unnecessary columns.
    2. Sum all count columns across the rows.
    3. Compute percent columns (e.g. Hispanic (%) = Hispanic / Enroll Total * 100).
    4. Copy the 'keep-as-is' columns from the first row.
    5. Standardize the district name using district_table_dict.
    6. Create a new 'District Label' that includes the county name.
    """

    # Delete Unneeded Columns
    merge_pair = unified_pair.drop(columns=delete_cols)

    # Sum Count Columns
    count_sums = merge_pair[count_cols].sum()

    # Calculate Percent Columns
    for percent_col, count_col in percent_cols_dict.items():
        count_sums[percent_col] = (count_sums[count_col] / count_sums['Enroll Total']) * 100

    # Convert Series to DataFrame; result is the final merged row
    result = count_sums.to_frame().T

    # Add Keep-As-Is Columns (first row values)
    result[keep_cols] = merge_pair[keep_cols].iloc[0]

    # Add District Name from expense_df's name, which was used in Unified Pair
    result['District Name'] = merge_pair['District Name'].iloc[0]

    # Update District Name using district_table_dict
    for non_unified, unified in district_table_dict.items():
        if non_unified in result['District Name'].values:
            result['District Name'] = unified

    # Create Unique District Label
    result['District Label'] = (
        result['District Name'].astype(str) + " (" + merge_pair['County Name'].iloc[0] + ")"
    )

    return result

# Process the 5 unified pairs
merged_pairs = [
    merge_unified_pairs(
        pair,
        KEEP_AS_IS_COLUMNS,
        DELETE_COLUMNS,
        COUNT_COLUMNS,
        PERCENT_COLUMNS_DICT,
        district_table_dict
    )
    for pair in unified_pair_list
]

# Display results
for i, merged_pair in enumerate(merged_pairs, 1):
    print(f"Merged Pair {i}:")
    display(merged_pair)

Arena Union Elementary
Santa Cruz City Elementary
Petaluma City Elementary
Santa Rosa Elementary
Modesto City Elementary
Merged Pair 1:
/tmp/ipykernel_729/1848033040.py:46: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  result[keep_cols] = merge_pair[keep_cols].iloc[0]
/tmp/ipykernel_729/1848033040.py:46: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  result[keep_cols] = merge_pair[keep_cols].iloc[0]
/tmp/ipykernel_729/1848033040.py:46: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To a

,Enroll Total,Enroll Charter,Enroll Non Charter,African American,American Indian,Asian,Filipino,Hispanic,Pacific Islander,White,...,District Code,County Name,District Type,Grade Low,Grade High,Grade Low Census,Grade High Census,Assistance Status,District Name,District Label
0,430.0,66.0,364.0,0.0,28.0,1.0,0.0,245.0,1.0,137.0,...,76349,Mendocino,Elementary,KG,12,KG,8,General Assistance,Arena Union Elementary/Point Arena Joint Union...,Arena Union Elementary/Point Arena Joint Union...


Merged Pair 2:


,Enroll Total,Enroll Charter,Enroll Non Charter,African American,American Indian,Asian,Filipino,Hispanic,Pacific Islander,White,...,District Code,County Name,District Type,Grade Low,Grade High,Grade Low Census,Grade High Census,Assistance Status,District Name,District Label
0,6272.0,116.0,6156.0,72.0,11.0,141.0,29.0,2537.0,13.0,2963.0,...,40261,Santa Cruz,Elementary,KG,5,KG,5,General Assistance,Santa Cruz City Elementary/High,Santa Cruz City Elementary/High (Santa Cruz)


Merged Pair 3:


,Enroll Total,Enroll Charter,Enroll Non Charter,African American,American Indian,Asian,Filipino,Hispanic,Pacific Islander,White,...,District Code,County Name,District Type,Grade Low,Grade High,Grade Low Census,Grade High Census,Assistance Status,District Name,District Label
0,7388.0,1325.0,6063.0,82.0,48.0,177.0,45.0,2593.0,28.0,3923.0,...,40246,Sonoma,Elementary,KG,12,KG,6,NaN,Petaluma City Elementary/Joint Union High,Petaluma City Elementary/Joint Union High (Son...


Merged Pair 4:


,Enroll Total,Enroll Charter,Enroll Non Charter,African American,American Indian,Asian,Filipino,Hispanic,Pacific Islander,White,...,District Code,County Name,District Type,Grade Low,Grade High,Grade Low Census,Grade High Census,Assistance Status,District Name,District Label
0,14798.0,1938.0,12860.0,265.0,89.0,586.0,145.0,8709.0,116.0,4113.0,...,40253,Sonoma,Elementary,KG,8,KG,6,NaN,Santa Rosa City Schools,Santa Rosa City Schools (Sonoma)


Merged Pair 5:


,Enroll Total,Enroll Charter,Enroll Non Charter,African American,American Indian,Asian,Filipino,Hispanic,Pacific Islander,White,...,District Code,County Name,District Type,Grade Low,Grade High,Grade Low Census,Grade High Census,Assistance Status,District Name,District Label
0,31431.0,1754.0,29677.0,771.0,87.0,1378.0,259.0,21296.0,155.0,4999.0,...,40717,Stanislaus,Elementary,KG,12,KG,8,Differentiated Assistance,Modesto City Schools,Modesto City Schools (Stanislaus)


#### 2.5.8. Add a `'District Label'` Column to the Rest of `district_data`

In [17]:
# Add `District Label` column to the rest of `district_data` that doesn't have unified pairs
# `District Label` = "District Name (County)"
district_data['District Label'] = (
    district_data['District Name'].astype(str)
    + " (" + district_data['County Name'] + ")"
)

# Display the first few rows of the updated `district_data` DataFrame
district_data.head()

,Fed ID,District Code,CDS Code,County Name,District Name,District Type,Grade Low,Grade High,Grade Low Census,Grade High Census,...,Foster (%),Homeless,Homeless (%),Migrant,Migrant (%),Students with Disabilities,Students with Disabilities (%),Socioeconomically Disadvantaged,Socioeconomically Disadvantaged (%),District Label
0,601770,61119,1611190000000,Alameda,Alameda Unified,Unified,KG,12,KG,12,...,0.3,91,0.9,0,0.0,1286,12.2,4035,38.2,Alameda Unified (Alameda)
1,601860,61127,1611270000000,Alameda,Albany City Unified,Unified,KG,12,KG,12,...,0.0,24,0.7,0,0.0,320,9.0,1122,31.4,Albany City Unified (Alameda)
2,604740,61143,1611430000000,Alameda,Berkeley Unified,Unified,KG,12,KG,12,...,0.4,150,1.7,0,0.0,1092,12.0,2508,27.6,Berkeley Unified (Alameda)
3,607800,61150,1611500000000,Alameda,Castro Valley Unified,Unified,KG,12,KG,12,...,0.1,135,1.4,0,0.0,1048,11.0,3686,38.8,Castro Valley Unified (Alameda)
4,612630,61168,1611680000000,Alameda,Emery Unified,Unified,KG,12,KG,12,...,0.2,13,2.2,0,0.0,75,12.5,327,54.5,Emery Unified (Alameda)


#### 2.5.9. Add the Newly Unified Rows to `district_data` and Drop the Old, Non-unified Rows

In [18]:
# Use .isin() to filter out the rows with the district codes for unified pairs from the `district_data` DataFrame
district_data = district_data[~district_data['District Code'].isin([76349, 40261, 40246, 40253, 40717])]

# Add the newly merged pairs to the `district_data` DataFrame
district_data = pd.concat([district_data, merged_pairs[0], merged_pairs[1], merged_pairs[2], merged_pairs[3], merged_pairs[4]], ignore_index=True)

# Display the shape and first few rows of the updated `district_data` DataFrame
print(district_data.shape)
display(district_data.tail())

(933, 46)


,Fed ID,District Code,CDS Code,County Name,District Name,District Type,Grade Low,Grade High,Grade Low Census,Grade High Census,...,Foster (%),Homeless,Homeless (%),Migrant,Migrant (%),Students with Disabilities,Students with Disabilities (%),Socioeconomically Disadvantaged,Socioeconomically Disadvantaged (%),District Label
928,NaN,76349,NaN,Mendocino,Arena Union Elementary/Point Arena Joint Union...,Elementary,KG,12,KG,8,...,0.930233,41.0,9.534884,2.0,0.465116,64.0,14.883721,243.0,56.511628,Arena Union Elementary/Point Arena Joint Union...
929,NaN,40261,NaN,Santa Cruz,Santa Cruz City Elementary/High,Elementary,KG,5,KG,5,...,0.223214,36.0,0.573980,34.0,0.542092,918.0,14.636480,2304.0,36.734694,Santa Cruz City Elementary/High (Santa Cruz)
930,NaN,40246,NaN,Sonoma,Petaluma City Elementary/Joint Union High,Elementary,KG,12,KG,6,...,0.257174,28.0,0.378993,11.0,0.148890,1309.0,17.717921,3326.0,45.018950,Petaluma City Elementary/Joint Union High (Son...
931,NaN,40253,NaN,Sonoma,Santa Rosa City Schools,Elementary,KG,8,KG,6,...,0.459522,283.0,1.912421,321.0,2.169212,2566.0,17.340181,7541.0,50.959589,Santa Rosa City Schools (Sonoma)
932,NaN,40717,NaN,Stanislaus,Modesto City Schools,Elementary,KG,12,KG,8,...,0.251344,281.0,0.894022,190.0,0.604499,3995.0,12.710381,24229.0,77.086316,Modesto City Schools (Stanislaus)


#### 2.5.10. Combine `district_data` with `expense_data`

In [19]:
# Use merge to combine `district_data` and `expense_data` on the 'District Code' column
district_and_expenses = district_data.merge(expense_data, how='inner', on='District Code')

# Display the first few rows of the combined DataFrame
district_and_expenses.head()

,Fed ID,District Code,CDS Code,County Name,District Name_x,District Type,Grade Low,Grade High,Grade Low Census,Grade High Census,...,Students with Disabilities,Students with Disabilities (%),Socioeconomically Disadvantaged,Socioeconomically Disadvantaged (%),District Label,District Name_y,EDP 365,Expense ADA,Expense per ADA,LEA Type
0,601770.0,61119,1.611190e+12,Alameda,Alameda Unified,Unified,KG,12,KG,12,...,1286.0,12.2,4035.0,38.2,Alameda Unified (Alameda),Alameda Unified,1.550948e+08,8567.86,18101.93,Unified
1,601860.0,61127,1.611270e+12,Alameda,Albany City Unified,Unified,KG,12,KG,12,...,320.0,9.0,1122.0,31.4,Albany City Unified (Alameda),Albany City Unified,6.149090e+07,3435.41,17899.14,Unified
2,604740.0,61143,1.611430e+12,Alameda,Berkeley Unified,Unified,KG,12,KG,12,...,1092.0,12.0,2508.0,27.6,Berkeley Unified (Alameda),Berkeley Unified,2.205508e+08,8572.17,25728.70,Unified
3,607800.0,61150,1.611500e+12,Alameda,Castro Valley Unified,Unified,KG,12,KG,12,...,1048.0,11.0,3686.0,38.8,Castro Valley Unified (Alameda),Castro Valley Unified,1.424913e+08,8991.52,15847.30,Unified
4,612630.0,61168,1.611680e+12,Alameda,Emery Unified,Unified,KG,12,KG,12,...,75.0,12.5,327.0,54.5,Emery Unified (Alameda),Emery Unified,1.586300e+07,554.70,28597.44,Unified


### 2.6 Drop District Name_x (from the District DF) & Use District Name_y (from the Expense DF) as Authoritative District Name

In [20]:
district_and_expenses = district_and_expenses.drop(columns=['District Name_x']).rename(columns={'District Name_y':'District Name'})

# Display the number of unique district codes in the combined DataFrame
district_and_expenses['District Code'].nunique()

933

### 2.7 Cleaning the 'Locale' Column

In [21]:
# Change the data type of the 'Locale' column in the DataFrame to str
district_and_expenses['Locale'] = district_and_expenses['Locale'].astype(str)

# Display the previous DataFrame 'Locale' column value counts
print(district_and_expenses['Locale'].value_counts())

# Create a function to assign the location type to each DataFrame row
def assign_location_type(locale):
    """
    Updates the row 'Locale' column value by removing the locale code (e.g., '21') and location subtype 
    (e.g., 'Large'). If the input is 'nan', returns 'Not Reported'.

    Args:
        locale (str): The 'Locale' column string from the DataFrame row (e.g., 'Urban 21 Large').

    Returns:
        str: The extracted location type, or 'Not Reported' if input is 'nan'.
    """
    if locale == 'nan':
        return 'Not Reported'
    else:
        return locale.split()[2].strip(',')

# Apply the 'assign_location_type' function to update each value in the 'Locale' column
district_and_expenses['Locale'] = district_and_expenses['Locale'].apply(assign_location_type)

# Display the new DataFrame 'Locale' column value counts
print(district_and_expenses['Locale'].value_counts())

Locale
21 - Suburban, Large      228
41 - Rural, Fringe        146
42 - Rural, Distant       136
32 - Town, Distant         67
43 - Rural, Remote         60
31 - Town, Fringe          57
11 - City, Large           49
12 - City, Midsize         43
13 - City, Small           41
22 - Suburban, Midsize     38
33 - Town, Remote          33
23 - Suburban, Small       30
nan                         5
Name: count, dtype: int64
Locale
Rural           342
Suburban        296
Town            157
City            133
Not Reported      5
Name: count, dtype: int64


### 2.8 Looking into Outliers - Discrepancies between Total Enrollment & ADA

We are going to look into the possible presence of outliers between the `'Expense ADA'` (count day attendance) & `'Enroll Total'` (total school enrollment) columns and filter out outliers that we deem to be too extreme.

In [22]:
# Explore Total Enrollment vs. ADA and filter outliers
# (|ADA - Enrollment| / ADA >= 50)
# 5000% Difference

# Store the filtered DataFrame that has rows to keep
district_and_expenses_no_out = district_and_expenses[
    ~((abs(district_and_expenses['Expense ADA'] - district_and_expenses['Enroll Total']) / district_and_expenses['Expense ADA'] >= 50))]

# Create a column to show the decimal difference between 'Expense ADA' and 'Enroll Total'
district_and_expenses['Decimal Difference'] = (abs(district_and_expenses['Expense ADA'] - district_and_expenses['Enroll Total']) / district_and_expenses['Expense ADA'])

# Display the number of rows with < 5000% difference
print(district_and_expenses_no_out.shape[0])

# Store the filtered DataFrame that has outlier rows to possibly drop
district_and_expenses_out = district_and_expenses[
    ((abs(district_and_expenses['Expense ADA'] - district_and_expenses['Enroll Total']) / district_and_expenses['Expense ADA'] >= 50))]

# Display the outlier rows to possibly drop
display(district_and_expenses_out)

928


,Fed ID,District Code,CDS Code,County Name,District Type,Grade Low,Grade High,Grade Low Census,Grade High Census,Assistance Status,...,Students with Disabilities (%),Socioeconomically Disadvantaged,Socioeconomically Disadvantaged (%),District Label,District Name,EDP 365,Expense ADA,Expense per ADA,LEA Type,Decimal Difference
192,623820.0,63628,1.563628e+13,Kern,Unified,KG,12,KG,12,Differentiated Assistance,...,10.1,7075.0,48.7,Maricopa Unified (Kern),Maricopa Unified,7810924.74,281.45,27752.44,Unified,50.643276
530,628950.0,67827,3.667827e+13,San Bernardino,Elementary,KG,12,KG,6,General Assistance,...,13.4,4794.0,83.8,Oro Grande (San Bernardino),Oro Grande Elementary,11112406.38,79.93,139026.73,Elementary,70.612661
552,610710.0,68049,3.768049e+13,San Diego,Elementary,KG,12,KG,8,General Assistance,...,10.5,4395.0,37.7,Dehesa Elementary (San Diego),Dehesa Elementary,4231109.69,227.51,18597.47,Elementary,50.281262
578,637680.0,68403,3.768403e+13,San Diego,Elementary,KG,12,KG,8,General Assistance,...,14.4,2644.0,68.1,Spencer Valley Elementary (San Diego),Spencer Valley Elementary,1594837.91,44.88,35535.60,Elementary,85.564171
596,627030.0,68627,3.968627e+13,San Joaquin,Elementary,KG,12,KG,8,General Assistance,...,12.7,4379.0,63.6,New Jerusalem Elementary (San Joaquin),New Jerusalem Elementary,5146020.92,13.45,382603.79,Elementary,511.118959


After looking through the output data, we noticed that `'New Jerusalem Elementary'` had a proportional difference of ~51000% between its `'Expense ADA'` (count day attendance) & `'Enroll Total'` (total school enrollment).

Due to this extreme discrepancy, we decided to omit `'New Jerusalmen Elementary'` from our dataset.

In [23]:
# Drop the row with 'New Jerusalem Elementary' data
district_and_expenses = district_and_expenses[district_and_expenses['District Name'] != 'New Jerusalem Elementary']

# Display the new shape of the DataFrame
district_and_expenses.shape

(932, 51)

## 3.0. Export the Cleaned Data into a CSV file

To organize our workflow, we decided to export our clean data to a CSV for easy reference in our other notebooks.

In [24]:
# Write district_and_expenses to a CSV file named 'district_and_expenses.csv'
district_and_expenses.to_csv('district_and_expenses.csv', index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=246b06f0-3e45-45e3-acef-efea2bae7701' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>